In [1]:
# should already be activated, but just a reminder where ya are :)  
# !conda activate wool_sucking_nn

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats 

%matplotlib inline

In [2]:
# import data
excel_file = pd.read_excel('..\Data\cleaned_and_split_data.xlsx', sheet_name=None)

# Unpack the dataframes into separate variables
X_train = excel_file['X_train']
X_test = excel_file['X_test']
y_train = excel_file['y_train']
y_test = excel_file['y_test']

In [3]:
print(X_train.shape)
X_train.head()

(3965, 7)


,Neuter_status,Breed_group,Aggression_owner,Aggression_cats,Shyness_novel,Shyness_strangers,Grooming
0,1,SIB,1,2,4,4,3
1,1,HCS,1,2,4,4,2
2,0,ORI,1,1,1,1,1
3,1,MCO,1,1,2,2,3
4,1,HCS,1,1,2,1,2


In [4]:
print(X_test.shape)
X_test.head()

(1700, 7)


,Neuter_status,Breed_group,Aggression_owner,Aggression_cats,Shyness_novel,Shyness_strangers,Grooming
0,1,SIB,1,3,3,1,1
1,1,MCO,1,1,2,1,1
2,1,RUS,1,1,4,4,1
3,0,EUR,2,2,2,2,1
4,1,NFO,1,2,3,2,1


In [5]:
print(y_train.shape)
y_train.head()

(3965, 1)


,Wool_sucking_binary
0,0
1,0
2,0
3,0
4,0


In [6]:
print(y_test.shape)
y_test.head()

(1700, 1)


,Wool_sucking_binary
0,1
1,1
2,1
3,0
4,1


## To record model metrics

In [7]:
# dataframe for scores amongst models
master_scores = pd.DataFrame(columns=['Model','Recall', 'F1', 'Precision', 'Accuracy'])

In [8]:
# add_to_master(model_description, y_true, y_predicted, binary=True)
def add_to_master(model_description, y_test, y_pred, binary=True):
    '''
    Adds a new row to the running score DataFrame `master_scores` 
    and functions for adding both binary and multiclass scores. 
    '''
    global master_scores
    if binary == True:
        # Evalution Metrics
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        # add to master_scores
        new_row = {
            'Model': model_description,
            'Recall': round(recall,3), 
            'F1': round(f1, 3), 
            'Precision': round(precision, 3), 
            'Accuracy': round(accuracy, 3)}
        
        master_scores = master_scores.append(new_row, ignore_index=True)   
    
    else:
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='macro') # I chose macro since we have less of the sucklers 
        recall = recall_score(y_test, y_pred, average='macro')       # and I want their accuracy to be meaningful
        f1 = f1_score(y_test, y_pred, average='macro')


        # add to master_scores
        new_row = {
            'Model': model_description,
            'Recall': round(recall,3), 
            'F1': round(f1, 3), 
            'Precision': round(precision, 3), 
            'Accuracy': round(accuracy, 3)}
        master_scores = master_scores.append(new_row, ignore_index=True)
    return master_scores

# Prep for tensorflow.keras

In [9]:
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import accuracy_score,precision_score, recall_score, f1_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
# for reproducibility 
tf.random.set_seed(42)
np.random.seed(42)

In [10]:
# one-hot breed 
X_train_encoded = pd.get_dummies(X_train, columns=['Breed_group'])
X_test_encoded = pd.get_dummies(X_test, columns=['Breed_group'])

In [11]:
X_train_encoded

,Neuter_status,Aggression_owner,Aggression_cats,Shyness_novel,Shyness_strangers,Grooming,Breed_group_ABY,Breed_group_BEN,Breed_group_BRI,Breed_group_BUR,...,Breed_group_MCO,Breed_group_NFO,Breed_group_ORI,Breed_group_PER,Breed_group_RAG,Breed_group_RUS,Breed_group_SBI,Breed_group_SIB,Breed_group_TUV,Breed_group_other
0,1,1,2,4,4,3,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,1,1,2,4,4,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,1,1,1,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,1,1,1,2,2,3,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,1,1,1,2,1,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3960,1,1,3,1,1,2,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3961,1,2,2,2,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3962,1,1,1,3,4,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3963,1,1,1,4,4,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Plain Jane

In [12]:
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(len(X_train_encoded.columns),)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

In [13]:
def recall(y_true, y_pred):
    true_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true * y_pred, 0, 1)))
    possible_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + tf.keras.backend.epsilon())
    return recall

tf.keras.utils.get_custom_objects()['recall'] = recall

In [14]:
model.compile(optimizer='adam', 
             loss='binary_crossentropy',
             metrics=['accuracy', 'recall'])

In [15]:
model.fit(X_train_encoded, y_train, epochs=10, batch_size=32)

Epoch 1/10
124/124 [==============================] - 1s 957us/step - loss: 0.6292 - accuracy: 0.6827 - recall: 0.0133
Epoch 2/10
124/124 [==============================] - 0s 957us/step - loss: 0.6056 - accuracy: 0.6845 - recall: 0.1046
Epoch 3/10
124/124 [==============================] - 0s 916us/step - loss: 0.5946 - accuracy: 0.6883 - recall: 0.1757
Epoch 4/10
124/124 [==============================] - 0s 941us/step - loss: 0.5903 - accuracy: 0.6878 - recall: 0.1451
Epoch 5/10
124/124 [==============================] - 0s 924us/step - loss: 0.5851 - accuracy: 0.6974 - recall: 0.1734
Epoch 6/10
124/124 [==============================] - 0s 908us/step - loss: 0.5825 - accuracy: 0.6963 - recall: 0.2081
Epoch 7/10
124/124 [==============================] - 0s 900us/step - loss: 0.5827 - accuracy: 0.6926 - recall: 0.2124
Epoch 8/10
124/124 [==============================] - 0s 908us/step - loss: 0.5770 - accuracy: 0.7019 - recall: 0.2071
Epoch 9/10
124/124 [============================

In [16]:
test_loss, test_acc, test_recall = model.evaluate(X_test_encoded, y_test)
print('Test Accuracy:', test_acc)
print('Test Loss:', test_loss)
print('Test Recall:', test_recall)

54/54 [==============================] - 0s 734us/step - loss: 0.5865 - accuracy: 0.6800 - recall: 0.2685
Test Accuracy: 0.6800000071525574
Test Loss: 0.5864678025245667
Test Recall: 0.26853251457214355


In [17]:
y_pred = model.predict(X_test_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
# add_to_master(model_description, y_true, y_predicted, binary=True)
add_to_master("1 hidden, 64 nodes per, adam", y_test, y_pred)
cm

54/54 [==============================] - 0s 602us/step


C:\Users\gwent\AppData\Local\Temp\ipykernel_14392\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


array([[1005,  154],
       [ 390,  151]], dtype=int64)

In [18]:
master_scores

,Model,Recall,F1,Precision,Accuracy
0,"1 hidden, 64 nodes per, adam",0.279,0.357,0.495,0.68


Not doing too hot, but also seems to be doing better than initial models in Cat-pstone 1. 

## Adagrad optimizer

In [19]:
model2 = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(len(X_train_encoded.columns),)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

In [20]:
# Adagrad better for when feataures vary in magnitude and frequency 
optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.01)
model2.compile(optimizer=optimizer, 
             loss='binary_crossentropy',
             metrics=['accuracy', 'recall'])

In [21]:
model2.fit(X_train_encoded, y_train, epochs=10, batch_size=32)

Epoch 1/10
124/124 [==============================] - 0s 811us/step - loss: 0.6373 - accuracy: 0.6810 - recall: 0.0000e+00
Epoch 2/10
124/124 [==============================] - 0s 835us/step - loss: 0.6236 - accuracy: 0.6815 - recall: 0.0019
Epoch 3/10
124/124 [==============================] - 0s 819us/step - loss: 0.6170 - accuracy: 0.6858 - recall: 0.0418
Epoch 4/10
124/124 [==============================] - 0s 819us/step - loss: 0.6130 - accuracy: 0.6858 - recall: 0.0432
Epoch 5/10
124/124 [==============================] - 0s 819us/step - loss: 0.6097 - accuracy: 0.6888 - recall: 0.0713
Epoch 6/10
124/124 [==============================] - 0s 860us/step - loss: 0.6069 - accuracy: 0.6868 - recall: 0.0932
Epoch 7/10
124/124 [==============================] - 0s 868us/step - loss: 0.6046 - accuracy: 0.6895 - recall: 0.0945
Epoch 8/10
124/124 [==============================] - 0s 835us/step - loss: 0.6029 - accuracy: 0.6923 - recall: 0.1032
Epoch 9/10
124/124 [========================

In [22]:
test_loss, test_acc, test_recall = model2.evaluate(X_test_encoded, y_test)
print('Test Accuracy:', test_acc)
print('Test Loss:', test_loss)
print('Test Recall:', test_recall)

54/54 [==============================] - 0s 715us/step - loss: 0.6031 - accuracy: 0.6865 - recall: 0.1331
Test Accuracy: 0.6864705681800842
Test Loss: 0.6031489372253418
Test Recall: 0.13311810791492462


In [23]:
y_pred = model2.predict(X_test_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
add_to_master("1 hidden, 64 nodes per, adagrad", y_test, y_pred)
cm

54/54 [==============================] - 0s 640us/step


C:\Users\gwent\AppData\Local\Temp\ipykernel_14392\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


array([[1092,   67],
       [ 466,   75]], dtype=int64)

## Scaled with Plain Jane and Adagrad

In [24]:
scaler = StandardScaler()
scaler.fit(X_train_encoded)

StandardScaler()

In [25]:
X_train_scaled_encoded = scaler.transform(X_train_encoded)
X_test_scaled_encoded = scaler.transform(X_test_encoded)

In [26]:
X_train_scaled_encoded.shape

(3965, 25)

In [27]:
# Plain Jane + scaled
model3 = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(25,)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model3.compile(optimizer='adam', 
             loss='binary_crossentropy',
             metrics=['accuracy', 'recall'])

model3.fit(X_train_scaled_encoded, y_train, epochs=10, batch_size=32)

test_loss, test_acc, test_recall = model3.evaluate(X_test_scaled_encoded, y_test)
print('Test Accuracy:', test_acc)
print('Test Loss:', test_loss)
print('Test Recall:', test_recall)

y_pred = model3.predict(X_test_scaled_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
add_to_master("1 hidden, 64 nodes per, adam, scaled", y_test, y_pred)
cm

Epoch 1/10
124/124 [==============================] - 0s 868us/step - loss: 0.6102 - accuracy: 0.6777 - recall: 0.1178
Epoch 2/10
124/124 [==============================] - 0s 900us/step - loss: 0.5882 - accuracy: 0.6918 - recall: 0.1488
Epoch 3/10
124/124 [==============================] - 0s 892us/step - loss: 0.5823 - accuracy: 0.6989 - recall: 0.2134
Epoch 4/10
124/124 [==============================] - 0s 908us/step - loss: 0.5791 - accuracy: 0.7011 - recall: 0.1787
Epoch 5/10
124/124 [==============================] - 0s 884us/step - loss: 0.5735 - accuracy: 0.7042 - recall: 0.2090
Epoch 6/10
124/124 [==============================] - 0s 916us/step - loss: 0.5728 - accuracy: 0.7044 - recall: 0.2274
Epoch 7/10
124/124 [==============================] - 0s 924us/step - loss: 0.5708 - accuracy: 0.7049 - recall: 0.2450
Epoch 8/10
124/124 [==============================] - 0s 900us/step - loss: 0.5669 - accuracy: 0.7052 - recall: 0.2385
Epoch 9/10
124/124 [============================

C:\Users\gwent\AppData\Local\Temp\ipykernel_14392\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


array([[977, 182],
       [390, 151]], dtype=int64)

In [28]:
# adagrad + scaled 
model4 = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(25,)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# Adagrad better for when feataures vary in magnitude and frequency 
optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.01)
model4.compile(optimizer=optimizer, 
             loss='binary_crossentropy',
             metrics=['accuracy', 'recall'])

model4.fit(X_train_scaled_encoded, y_train, epochs=10, batch_size=32)

test_loss, test_acc, test_recall = model4.evaluate(X_test_scaled_encoded, y_test)
print('Test Accuracy:', test_acc)
print('Test Loss:', test_loss)
print('Test Recall:', test_recall)

y_pred = model4.predict(X_test_scaled_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
add_to_master("1 hidden, 64 nodes per, adagrad, scaled", y_test, y_pred)
cm

Epoch 1/10
124/124 [==============================] - 0s 811us/step - loss: 0.6231 - accuracy: 0.6830 - recall: 0.0212
Epoch 2/10
124/124 [==============================] - 0s 876us/step - loss: 0.6010 - accuracy: 0.6910 - recall: 0.0654
Epoch 3/10
124/124 [==============================] - 0s 981us/step - loss: 0.5924 - accuracy: 0.6931 - recall: 0.1287
Epoch 4/10
124/124 [==============================] - 0s 941us/step - loss: 0.5882 - accuracy: 0.6941 - recall: 0.1370
Epoch 5/10
124/124 [==============================] - 0s 908us/step - loss: 0.5853 - accuracy: 0.7006 - recall: 0.1620
Epoch 6/10
124/124 [==============================] - 0s 949us/step - loss: 0.5833 - accuracy: 0.6999 - recall: 0.1818
Epoch 7/10
124/124 [==============================] - 0s 916us/step - loss: 0.5817 - accuracy: 0.6971 - recall: 0.1746
Epoch 8/10
124/124 [==============================] - 0s 868us/step - loss: 0.5804 - accuracy: 0.7004 - recall: 0.1917
Epoch 9/10
124/124 [============================

C:\Users\gwent\AppData\Local\Temp\ipykernel_14392\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


array([[1072,   87],
       [ 438,  103]], dtype=int64)

In [29]:
master_scores.sort_values('Recall', ascending=False)

,Model,Recall,F1,Precision,Accuracy
0,"1 hidden, 64 nodes per, adam",0.279,0.357,0.495,0.680
2,"1 hidden, 64 nodes per, adam, scaled",0.279,0.346,0.453,0.664
3,"1 hidden, 64 nodes per, adagrad, scaled",0.190,0.282,0.542,0.691
1,"1 hidden, 64 nodes per, adagrad",0.139,0.220,0.528,0.686


In [30]:
def create_mlp(nodes, optimizer, X_train):
    # Define the model architecture
    model = keras.Sequential([
    keras.layers.Dense(nodes, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(nodes, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

    model.compile(optimizer=optimizer, 
             loss='binary_crossentropy',
             metrics=['accuracy', 'recall'])
    
    return model

In [31]:
model5 = create_mlp(100, 'adam', X_train_scaled_encoded)

In [32]:
model5.fit(X_train_scaled_encoded, y_train, epochs=10, batch_size=32)

test_loss, test_acc, test_recall = model5.evaluate(X_test_scaled_encoded, y_test)
print('Test Accuracy:', test_acc)
print('Test Loss:', test_loss)
print('Test Recall:', test_recall)

y_pred = model5.predict(X_test_scaled_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
add_to_master("1 hidden, 100 nodes, adam", y_test, y_pred)
cm

Epoch 1/10
124/124 [==============================] - 0s 973us/step - loss: 0.6047 - accuracy: 0.6820 - recall: 0.1112
Epoch 2/10
124/124 [==============================] - 0s 997us/step - loss: 0.5856 - accuracy: 0.6956 - recall: 0.1908
Epoch 3/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5789 - accuracy: 0.6981 - recall: 0.2367
Epoch 4/10
124/124 [==============================] - 0s 997us/step - loss: 0.5758 - accuracy: 0.6979 - recall: 0.2002
Epoch 5/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5699 - accuracy: 0.7054 - recall: 0.2299
Epoch 6/10
124/124 [==============================] - 0s 989us/step - loss: 0.5701 - accuracy: 0.7049 - recall: 0.2390
Epoch 7/10
124/124 [==============================] - 0s 981us/step - loss: 0.5674 - accuracy: 0.7047 - recall: 0.2682
Epoch 8/10
124/124 [==============================] - 0s 981us/step - loss: 0.5628 - accuracy: 0.7130 - recall: 0.2709
Epoch 9/10
124/124 [==============================] 

C:\Users\gwent\AppData\Local\Temp\ipykernel_14392\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


array([[979, 180],
       [392, 149]], dtype=int64)

In [33]:
model6 = create_mlp(32, 'adam', X_train_scaled_encoded)
model6.fit(X_train_scaled_encoded, y_train, epochs=10, batch_size=32)

test_loss, test_acc, test_recall = model6.evaluate(X_test_scaled_encoded, y_test)
print('Test Accuracy:', test_acc)
print('Test Loss:', test_loss)
print('Test Recall:', test_recall)

y_pred = model6.predict(X_test_scaled_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
add_to_master("1 hidden, 32 nodes, adam", y_test, y_pred)
cm

Epoch 1/10
124/124 [==============================] - 0s 843us/step - loss: 0.6189 - accuracy: 0.6726 - recall: 0.1100
Epoch 2/10
124/124 [==============================] - 0s 860us/step - loss: 0.5946 - accuracy: 0.6923 - recall: 0.1084
Epoch 3/10
124/124 [==============================] - 0s 860us/step - loss: 0.5888 - accuracy: 0.6956 - recall: 0.1807
Epoch 4/10
124/124 [==============================] - 0s 860us/step - loss: 0.5842 - accuracy: 0.7009 - recall: 0.1627
Epoch 5/10
124/124 [==============================] - 0s 868us/step - loss: 0.5805 - accuracy: 0.6994 - recall: 0.1814
Epoch 6/10
124/124 [==============================] - 0s 900us/step - loss: 0.5782 - accuracy: 0.6994 - recall: 0.1912
Epoch 7/10
124/124 [==============================] - 0s 876us/step - loss: 0.5763 - accuracy: 0.7009 - recall: 0.2008
Epoch 8/10
124/124 [==============================] - 0s 860us/step - loss: 0.5737 - accuracy: 0.7059 - recall: 0.2122
Epoch 9/10
124/124 [============================

C:\Users\gwent\AppData\Local\Temp\ipykernel_14392\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


array([[1026,  133],
       [ 422,  119]], dtype=int64)

In [34]:
model7 = create_mlp(250, 'adam', X_train_scaled_encoded)
model7.fit(X_train_scaled_encoded, y_train, epochs=10, batch_size=32)

test_loss, test_acc, test_recall = model7.evaluate(X_test_scaled_encoded, y_test)
print('Test Accuracy:', test_acc)
print('Test Loss:', test_loss)
print('Test Recall:', test_recall)

y_pred = model7.predict(X_test_scaled_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
add_to_master("1 hidden, 250 nodes, adam", y_test, y_pred)
cm

Epoch 1/10
124/124 [==============================] - 1s 1ms/step - loss: 0.6075 - accuracy: 0.6835 - recall: 0.1611
Epoch 2/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5875 - accuracy: 0.6963 - recall: 0.2258
Epoch 3/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5795 - accuracy: 0.7009 - recall: 0.2328
Epoch 4/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5776 - accuracy: 0.7016 - recall: 0.2061
Epoch 5/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5687 - accuracy: 0.7112 - recall: 0.2430
Epoch 6/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5705 - accuracy: 0.7105 - recall: 0.2699
Epoch 7/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5675 - accuracy: 0.7047 - recall: 0.2743
Epoch 8/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5614 - accuracy: 0.7117 - recall: 0.2695
Epoch 9/10
124/124 [==============================] - 0s 1ms/ste

C:\Users\gwent\AppData\Local\Temp\ipykernel_14392\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


array([[979, 180],
       [387, 154]], dtype=int64)

In [35]:
# definitely overfits
model8 = create_mlp(300, 'adam', X_train_scaled_encoded)
model8.fit(X_train_scaled_encoded, y_train, epochs=10, batch_size=32)

test_loss, test_acc, test_recall = model8.evaluate(X_test_scaled_encoded, y_test)
print('Test Accuracy:', test_acc)
print('Test Loss:', test_loss)
print('Test Recall:', test_recall)

y_pred = model8.predict(X_test_scaled_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
add_to_master("1 hidden, 300 nodes, adam", y_test, y_pred)
cm

Epoch 1/10
124/124 [==============================] - 0s 1ms/step - loss: 0.6088 - accuracy: 0.6802 - recall: 0.1688
Epoch 2/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5888 - accuracy: 0.6968 - recall: 0.2117
Epoch 3/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5802 - accuracy: 0.6979 - recall: 0.2268
Epoch 4/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5781 - accuracy: 0.6986 - recall: 0.1981
Epoch 5/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5690 - accuracy: 0.7064 - recall: 0.2432
Epoch 6/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5714 - accuracy: 0.7079 - recall: 0.2591
Epoch 7/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5683 - accuracy: 0.7084 - recall: 0.2676
Epoch 8/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5620 - accuracy: 0.7107 - recall: 0.2675
Epoch 9/10
124/124 [==============================] - 0s 1ms/ste

C:\Users\gwent\AppData\Local\Temp\ipykernel_14392\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


array([[979, 180],
       [390, 151]], dtype=int64)

In [36]:
master_scores.sort_values('Recall', ascending=False)

,Model,Recall,F1,Precision,Accuracy
6,"1 hidden, 250 nodes, adam",0.285,0.352,0.461,0.666
0,"1 hidden, 64 nodes per, adam",0.279,0.357,0.495,0.680
2,"1 hidden, 64 nodes per, adam, scaled",0.279,0.346,0.453,0.664
7,"1 hidden, 300 nodes, adam",0.279,0.346,0.456,0.665
4,"1 hidden, 100 nodes, adam",0.275,0.343,0.453,0.664
5,"1 hidden, 32 nodes, adam",0.220,0.300,0.472,0.674
3,"1 hidden, 64 nodes per, adagrad, scaled",0.190,0.282,0.542,0.691
1,"1 hidden, 64 nodes per, adagrad",0.139,0.220,0.528,0.686


Okay so we are pretty similar in performance to the logistic regression models from cat-pstone 1 on every metric except recall (and the part it plays in F1, of course). Let's see if over or undersampling with the top performing model architecture (model1). 

In [37]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

In [38]:
rus = RandomUnderSampler()
X_resampled_rus, y_resampled_rus = rus.fit_resample(X_train_scaled_encoded, y_train)

ros = RandomOverSampler()
X_resampled_ros, y_resampled_ros = ros.fit_resample(X_train_scaled_encoded, y_train)

In [39]:
# RUS 
model9 = create_mlp(64, 'adam', X_train_scaled_encoded)
model9.fit(X_resampled_rus, y_resampled_rus, epochs=10, batch_size=32)

y_pred = model9.predict(X_test_scaled_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
add_to_master("RUS 1 hidden, 64 nodes, adam", y_test, y_pred)
cm

Epoch 1/10
79/79 [==============================] - 0s 857us/step - loss: 0.6777 - accuracy: 0.5677 - recall: 0.5536
Epoch 2/10
79/79 [==============================] - 0s 895us/step - loss: 0.6496 - accuracy: 0.6256 - recall: 0.6189
Epoch 3/10
79/79 [==============================] - 0s 921us/step - loss: 0.6418 - accuracy: 0.6355 - recall: 0.6324
Epoch 4/10
79/79 [==============================] - 0s 895us/step - loss: 0.6358 - accuracy: 0.6359 - recall: 0.6311
Epoch 5/10
79/79 [==============================] - 0s 895us/step - loss: 0.6327 - accuracy: 0.6466 - recall: 0.6368
Epoch 6/10
79/79 [==============================] - 0s 895us/step - loss: 0.6287 - accuracy: 0.6470 - recall: 0.6630
Epoch 7/10
79/79 [==============================] - 0s 895us/step - loss: 0.6230 - accuracy: 0.6557 - recall: 0.6529
Epoch 8/10
79/79 [==============================] - 0s 895us/step - loss: 0.6189 - accuracy: 0.6581 - recall: 0.6703
Epoch 9/10
79/79 [==============================] - 0s 933us/ste

C:\Users\gwent\AppData\Local\Temp\ipykernel_14392\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


array([[758, 401],
       [240, 301]], dtype=int64)

In [40]:
# ROS 
model10 = create_mlp(64, 'adam', X_train_scaled_encoded)
model10.fit(X_resampled_ros, y_resampled_ros, epochs=10, batch_size=32)

y_pred = model10.predict(X_test_scaled_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
add_to_master("ROS 1 hidden, 64 nodes, adam", y_test, y_pred)
cm

Epoch 1/10
169/169 [==============================] - 1s 855us/step - loss: 0.6649 - accuracy: 0.5910 - recall: 0.5849
Epoch 2/10
169/169 [==============================] - 0s 885us/step - loss: 0.6443 - accuracy: 0.6271 - recall: 0.6106
Epoch 3/10
169/169 [==============================] - 0s 873us/step - loss: 0.6361 - accuracy: 0.6413 - recall: 0.6451
Epoch 4/10
169/169 [==============================] - 0s 914us/step - loss: 0.6302 - accuracy: 0.6426 - recall: 0.6284
Epoch 5/10
169/169 [==============================] - 0s 885us/step - loss: 0.6218 - accuracy: 0.6587 - recall: 0.6446
Epoch 6/10
169/169 [==============================] - 0s 891us/step - loss: 0.6178 - accuracy: 0.6585 - recall: 0.6640
Epoch 7/10
169/169 [==============================] - 0s 885us/step - loss: 0.6129 - accuracy: 0.6580 - recall: 0.6602
Epoch 8/10
169/169 [==============================] - 0s 885us/step - loss: 0.6093 - accuracy: 0.6624 - recall: 0.6482
Epoch 9/10
169/169 [============================

C:\Users\gwent\AppData\Local\Temp\ipykernel_14392\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


array([[798, 361],
       [261, 280]], dtype=int64)

In [41]:
master_scores.sort_values('Recall', ascending=False)

,Model,Recall,F1,Precision,Accuracy
8,"RUS 1 hidden, 64 nodes, adam",0.556,0.484,0.429,0.623
9,"ROS 1 hidden, 64 nodes, adam",0.518,0.474,0.437,0.634
6,"1 hidden, 250 nodes, adam",0.285,0.352,0.461,0.666
0,"1 hidden, 64 nodes per, adam",0.279,0.357,0.495,0.680
2,"1 hidden, 64 nodes per, adam, scaled",0.279,0.346,0.453,0.664
7,"1 hidden, 300 nodes, adam",0.279,0.346,0.456,0.665
4,"1 hidden, 100 nodes, adam",0.275,0.343,0.453,0.664
5,"1 hidden, 32 nodes, adam",0.220,0.300,0.472,0.674
3,"1 hidden, 64 nodes per, adagrad, scaled",0.190,0.282,0.542,0.691
1,"1 hidden, 64 nodes per, adagrad",0.139,0.220,0.528,0.686


In [42]:
# RUS 
model11 = create_mlp(64, 'adam', X_train_scaled_encoded)
model11.fit(X_resampled_rus, y_resampled_rus, epochs=10, batch_size=25)

y_pred = model11.predict(X_test_scaled_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
add_to_master("RUS, adam, batch 25", y_test, y_pred)
cm

Epoch 1/10
101/101 [==============================] - 0s 848us/step - loss: 0.6716 - accuracy: 0.5927 - recall: 0.5684
Epoch 2/10
101/101 [==============================] - 0s 878us/step - loss: 0.6480 - accuracy: 0.6288 - recall: 0.6112
Epoch 3/10
101/101 [==============================] - 0s 898us/step - loss: 0.6403 - accuracy: 0.6375 - recall: 0.6403
Epoch 4/10
101/101 [==============================] - 0s 908us/step - loss: 0.6337 - accuracy: 0.6462 - recall: 0.6416
Epoch 5/10
101/101 [==============================] - 0s 898us/step - loss: 0.6301 - accuracy: 0.6462 - recall: 0.6368
Epoch 6/10
101/101 [==============================] - 0s 888us/step - loss: 0.6245 - accuracy: 0.6569 - recall: 0.6669
Epoch 7/10
101/101 [==============================] - 0s 938us/step - loss: 0.6213 - accuracy: 0.6573 - recall: 0.6545
Epoch 8/10
101/101 [==============================] - 0s 878us/step - loss: 0.6167 - accuracy: 0.6609 - recall: 0.6738
Epoch 9/10
101/101 [============================

C:\Users\gwent\AppData\Local\Temp\ipykernel_14392\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


array([[703, 456],
       [225, 316]], dtype=int64)

In [43]:
master_scores.sort_values('Recall', ascending=False).head()

,Model,Recall,F1,Precision,Accuracy
10,"RUS, adam, batch 25",0.584,0.481,0.409,0.599
8,"RUS 1 hidden, 64 nodes, adam",0.556,0.484,0.429,0.623
9,"ROS 1 hidden, 64 nodes, adam",0.518,0.474,0.437,0.634
6,"1 hidden, 250 nodes, adam",0.285,0.352,0.461,0.666
0,"1 hidden, 64 nodes per, adam",0.279,0.357,0.495,0.680


Well we've gotten it pretty close to the performance of the LR model. Is anything about it better? Run speed?

In [44]:
import time
# training time 
start = time.time()
model10.fit(X_resampled_ros, y_resampled_ros, epochs=10, batch_size=32)
end = time.time()
training_time = end - start

p_start = time.time()
y_pred = model10.predict(X_test_encoded)
p_end = time.time()
predict_time = p_end - p_start

Epoch 1/10
169/169 [==============================] - 0s 1ms/step - loss: 0.5968 - accuracy: 0.6737 - recall: 0.6706
Epoch 2/10
169/169 [==============================] - 0s 974us/step - loss: 0.5930 - accuracy: 0.6818 - recall: 0.6874
Epoch 3/10
169/169 [==============================] - 0s 956us/step - loss: 0.5912 - accuracy: 0.6866 - recall: 0.6986
Epoch 4/10
169/169 [==============================] - 0s 968us/step - loss: 0.5885 - accuracy: 0.6811 - recall: 0.6729
Epoch 5/10
169/169 [==============================] - 0s 997us/step - loss: 0.5824 - accuracy: 0.6946 - recall: 0.7002
Epoch 6/10
169/169 [==============================] - 0s 1ms/step - loss: 0.5812 - accuracy: 0.6902 - recall: 0.7065
Epoch 7/10
169/169 [==============================] - 0s 1ms/step - loss: 0.5782 - accuracy: 0.6874 - recall: 0.6915
Epoch 8/10
169/169 [==============================] - 0s 1ms/step - loss: 0.5770 - accuracy: 0.6922 - recall: 0.6864
Epoch 9/10
169/169 [==============================] - 0s

# Comparing to a Logistic Regression
In Cat-pstone 1 I had did not prune down the columns nearly as much. I want to see how it does with this new training set. 

In [53]:
from sklearn.linear_model import LogisticRegression

# fit model, + training time 
t_start = time.time()
log_balanced = LogisticRegression(class_weight='balanced').fit(X_train_encoded, (y_train.values.reshape(-1)))
t_end = time.time()
log_training = t_end - t_start

# predict w model, + prediction time 
p_start = time.time()
y_pred = log_balanced.predict(X_test_encoded)
p_end = time.time()
log_predict = p_end - p_start 

# add_to_master(model_description, y_true, y_predicted, binary=True)
add_to_master("class_weight='balanced', LogReg", y_test, y_pred)

In [54]:
# scaled and logistic regression
log_balanced_scaled = LogisticRegression(class_weight='balanced').fit(X_train_encoded, (y_train.values.reshape(-1)))
y_pred = log_balanced_scaled.predict(X_test_scaled_encoded)

# add_to_master(model_description, y_true, y_predicted, binary=True)
add_to_master("class_weight='balanced', LogReg, scaled", y_test, y_pred);

C:\Users\gwent\AppData\Local\Temp\ipykernel_14392\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)
C:\Users\gwent\anaconda3\envs\wool_sucking_nn\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
C:\Users\gwent\AppData\Local\Temp\ipykernel_14392\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


In [48]:
print(f"MLP training time: {training_time} \nMLP predict time: {predict_time} \nLogistic Regression training time: {log_training} \nLogistic Regression predict time: {log_predict}")
train_ratio = round(training_time / log_training, 1)
predict_ratio = round(predict_time / log_predict, 1)
print(f"MLP is {train_ratio} times slower at training than logistic regression. \nMLP is {predict_ratio} times slower at predicting than logistic regression." )
# AND I didn't even include the time spent doing a random undersampling for the MLP, which logistic reg didn't need 

MLP training time: 1.7623367309570312 
MLP predict time: 0.13364672660827637 
Logistic Regression training time: 0.029920339584350586 
Logistic Regression predict time: 0.000997304916381836
MLP is 58.9 times slower at training than logistic regression. 
MLP is 134.0 times slower at predicting than logistic regression.


In [ ]:
# for reference, cat-pstone 1 model was 0.622, 0.519, 0.446, 0.635
master_scores.sort_values('Recall', ascending=False).head()

Given that this model does not perform as well as the cat-pstone 1 model (marginally), I want to see how my best performing model architecture does on the dataset I used with cat-pstone 1 (with a lot more columns!). 

In [ ]:
from sklearn.model_selection import train_test_split
df = pd.read_csv('..\Data\cat_clean.csv')

In [ ]:
# binarize wool-sucking
df_bin = df.copy()
df_bin['ws_binary'] = df_bin['Wool_sucking'].replace([1, 2, 3, 4, 5, 6, 7], [1, 1, 1, 1, 1, 1, 1])
df_bin.drop(columns='Wool_sucking', inplace=True)

# Recode Behaviour_problem to binary
df_bin['Behaviour_problem'].replace([1, 2, 3], [0, 1, 1], inplace=True)

# encode breed group
df_bin_encoded = pd.get_dummies(df_bin, columns=['Breed_group'])

In [ ]:
X = df_bin_encoded.drop(columns=['ws_binary'])
y = df_bin_encoded['ws_binary']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42,stratify=y)

# making agg columns for training set 
X_train['agg_shy'] = (X_train['Shyness_novel'] + X_train['Shyness_strangers']) / 2
X_train['agg_aggress_people'] = (X_train['Aggression_owner'] + X_train['Aggression_stranger']) / 2 
X_train.drop(columns=['Shyness_novel', 'Shyness_strangers', 'Aggression_stranger', 'Aggression_owner'], inplace=True)
X_train.columns

# making agg columns for testing set
X_test['agg_shy'] = (X_test['Shyness_novel'] + X_test['Shyness_strangers']) / 2
X_test['agg_aggress_people'] = (X_test['Aggression_owner'] + X_test['Aggression_stranger']) / 2
X_test.drop(columns=['Shyness_novel', 'Shyness_strangers', 'Aggression_stranger', 'Aggression_owner'], inplace=True)
X_test.columns

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled_encoded = scaler.transform(X_train)
X_test_scaled_encoded = scaler.transform(X_test)

rus = RandomUnderSampler()
X_resampled_rus, y_resampled_rus = rus.fit_resample(X_train_scaled_encoded, y_train)

model12 = create_mlp(64, 'adam', X_resampled_rus)
model12.fit(X_resampled_rus, y_resampled_rus, epochs=10, batch_size=32)

y_pred = model12.predict(X_test_scaled_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
add_to_master("old data rus adam 64", y_test, y_pred)
cm

In [ ]:
# for reference, cat-pstone 1 model was 0.622, 0.519, 0.446, 0.635
# it really doesn't do any better than a weighted LR model, in either dataset
master_scores.sort_values('Recall', ascending=False)